# T1 SEMANTIC SEGMENTATION TRAINING

Small example on how to train Pointcept for semantic segmentation

## LIBRARIES

In [9]:
#IMPORT PACKAGES
from pathlib import Path
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('.')))
sys.path.insert(0, os.path.abspath(os.path.join('scripts')))
sys.path.insert(0, os.path.abspath(os.path.join('thirdparty', 'pointcept')))
print(sys.path)
import numpy as np
import laspy
from geomapi.utils import geometryutils as gmu
import torch

import context 


['/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts/thirdparty/pointcept', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts/scripts', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts/thirdparty/pointcept', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts/scripts', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts', '/home/mbassier/.conda/envs/pointcept/lib/python310.zip', '/home/mbassier/.conda/envs/pointcept/lib/python3.10', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/lib-dynload', '', '/home/mbassier/.local/lib/python3.10/site-packages', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/site-packages', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/site-packages/pointops-1.0-py3.10-linux-x86_64.egg', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/site-packages/pointgroup_ops-0.0.0-py3.10-linux-x86_64.egg', '/home/mbassier/.conda/envs/pointcept/lib/p

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## INPUT DATA CONVERSION

Preprocessing of input data

In [7]:
def handle_process(file_name, output_folder):
    
    coords = []
    scene_id = os.path.basename(file_name)

    name, ext = os.path.splitext(scene_id)
    
    if ext not in  [".las", ".laz"]:
        return

    # Read LAS/LAZ
    # populate dict
    las = laspy.read(file_name)
    print(list(las.point_format.dimension_names))

    pcd = gmu.las_to_pcd(las)
    pcd.estimate_normals()
    pcd.orient_normals_to_align_with_direction()
    
    coords = np.stack([las.x, las.y, las.z], axis=1)
    colors = np.stack([las.red, las.green, las.blue], axis=1).astype(np.uint8)
    normals = np.asarray(pcd.normals)
    verticality = np.nan_to_num(las.verticality)
    max = np.max(verticality)
    verticality = verticality / (max / 2.) - 1.
    
    save_dict = dict(coord=coords, color=colors, normal=normals, verticality=verticality, scene_id=scene_id, semantic_gt=las.labels.astype(int))

    torch.save(save_dict, os.path.join(output_folder, f"{name}.pth"))


training_las_path = Path(os.getcwd()).parents[0]/'data'/'t1_data_test'/'input'/'train.las'
training_output_folder = Path(os.getcwd()).parents[0]/'data'/'t1_data_test'/'train'

validation_las_path = Path(os.getcwd()).parents[0]/'data'/'t1_data_test'/'input'/'val.las'
validation_output_folder = Path(os.getcwd()).parents[0]/'data'/'t1_data_test'/'val'

os.makedirs(training_output_folder, exist_ok=True)
os.makedirs(validation_output_folder, exist_ok=True)

handle_process(training_las_path, training_output_folder)
handle_process(validation_las_path, validation_output_folder)

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue', 'labels', 'normal_change_rate', 'verticality', 'omnivariance']
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue', 'labels', 'normal_change_rate', 'verticality', 'omnivariance']


In [6]:
print(Path(os.getcwd()).parents[0]/'data'/'t1_data_test'/'input'/'train.las')

/home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1_data/input/train.las


## TRAINING

Training using Point Transformer V3

In [10]:
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.engines.train import TRAINERS
from pointcept.engines.launch import launch


def main_worker(cfg):
    cfg = default_setup(cfg)
    trainer = TRAINERS.build(dict(type=cfg.train.type, cfg=cfg))
    trainer.train()

config_path = Path(os.getcwd()).parents[0]/'data'/'t1_data_test'/'config.py' #Path(os.getcwd())/'data'/'t1_data'/'config.py'
save_path = Path(os.getcwd()).parents[0]/'data'/'t1_data_test' #Path(os.getcwd())/'data'/'t1_data'
weights = Path(os.getcwd()).parents[0]/'data'/'t1_data_test'/'model'/'model_best.pth' #Path(os.getcwd())/'data'/'t1_data'/'model'/'model_best.pth'

cfg = default_config_parser(str(config_path), {'save_path': str(save_path), 'weight': str(weights)})

launch(
    main_worker,
    num_gpus_per_machine=1,
    num_machines=1,
    machine_rank=0,
    dist_url='auto',
    cfg=(cfg,),
)

ModuleNotFoundError: No module named 'pointcept'